In [2]:
# Cell 1: importy i konfiguracja ścieżki do modułów projektu
import json
from pathlib import Path
import sys

import yaml  # type: ignore

# Upewniamy się, że katalog "src" (z modułami projektu) jest na ścieżce importów
src_path = Path("src").resolve()
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from axiomatic_kernel import (
    AxiomKernel,
    VariableSchema,
    AxiomDefinition,
    DecisionLogger,
)
from nl_rule_parser import build_axiom_from_nl, RuleParseError  # noqa: F401
from explanation_engine import DecisionExplainer, ExplanationConfig
from rules_io import load_ruleset_from_file  # noqa: F401
from ruleset_manager import RulesetRegistry, Environment
from rule_analytics import RuleAnalyticsEngine


In [3]:
# Cell: konfiguracja schematu AML, kernela i wczytanie rulesetu AML (governance)


# === 1) SCHEMA DOPASOWANA DO AML_DATA_GENERATOR (tylko pola używane w regułach) ===
schema_aml = [
    VariableSchema(
        "amount",
        "int",
        "Kwota transakcji w jednostkach minimalnych.",
    ),
    VariableSchema(
        "tx_count_24h",
        "int",
        "Liczba transakcji klienta w ostatnich 24h.",
    ),
    VariableSchema(
        "total_amount_24h",
        "int",
        "Łączna kwota transakcji klienta w ostatnich 24h.",
    ),
    VariableSchema(
        "tx_count_7d",
        "int",
        "Liczba transakcji klienta w ostatnich 7 dniach.",
    ),
    VariableSchema(
        "total_amount_7d",
        "int",
        "Łączna kwota transakcji klienta w ostatnich 7 dniach.",
    ),
    VariableSchema(
        "unique_counterparties_30d",
        "int",
        "Liczba unikalnych kontrahentów w ostatnich 30 dniach.",
    ),
    VariableSchema(
        "model_risk_score",
        "real",
        "Wynik modelu ryzyka (0–1).",
    ),
    VariableSchema(
        "is_pep",
        "bool",
        "Czy klient jest PEP.",
    ),
    VariableSchema(
        "on_sanctions_list",
        "bool",
        "Czy klient znajduje się na liście sankcyjnej.",
    ),
    VariableSchema(
        "is_suspicious",
        "bool",
        "Decyzja systemu – czy transakcja jest podejrzana.",
    ),
]

# === 2) Katalog na logi decyzji AML ===
logs_dir = Path("logs")
logs_dir.mkdir(exist_ok=True)

aml_logger = DecisionLogger(logs_dir / "aml_rules_demo.jsonl")

# === 3) Kernel AML oparty o AxiomKernel ===
aml_kernel = AxiomKernel(
    schema=schema_aml,
    decision_variable="is_suspicious",
    logger=aml_logger,
    rule_version="aml_rules_v1",  # zostanie nadpisane przez RulesetRegistry
)

# === 4) Ruleset AML + governance (RulesetRegistry) ===
rules_dir = Path("rules")
rules_dir.mkdir(exist_ok=True)

aml_rules_path = rules_dir / "aml_rules_v1.yaml"

# Jeśli plik z regułami AML jeszcze nie istnieje, tworzymy przykładowy.
if not aml_rules_path.exists():
    aml_rules_yaml = """ruleset_id: "aml_rules_v1"
version: "1.0.0"
description: "Reguły AML dopasowane do syntetycznego generatora aml_data_generator"

rules:
  - id: "aml.high_amount_core"
    text: "IF amount >= 15000 THEN is_suspicious = TRUE"
    description: "Wysokie kwoty pojedynczych transakcji (scenariusz high_amount)."
    enabled: true
    severity: "HIGH"
    tags: ["aml", "amount", "high_amount"]
    metadata:
      scenario: "high_amount"

  - id: "aml.velocity_24h_core"
    text: "IF tx_count_24h >= 10 THEN is_suspicious = TRUE"
    description: "Duża liczba transakcji w ciągu 24h (scenariusz velocity / structuring)."
    enabled: true
    severity: "HIGH"
    tags: ["aml", "velocity", "tx_24h"]
    metadata:
      scenario: "velocity"

  - id: "aml.velocity_7d_heavy"
    text: "IF tx_count_7d >= 50 AND total_amount_7d > 50000 THEN is_suspicious = TRUE"
    description: "Bardzo intensywna aktywność w 7 dni (wysoka liczba i łączna kwota)."
    enabled: true
    severity: "HIGH"
    tags: ["aml", "velocity", "tx_7d", "high_amount"]
    metadata:
      scenario: "velocity"

  - id: "aml.pep_high_risk_amount"
    text: "IF is_pep == TRUE AND amount >= 2000 THEN is_suspicious = TRUE"
    description: "Transakcje PEP powyżej progu kwotowego (scenariusz pep_high_risk)."
    enabled: true
    severity: "HIGH"
    tags: ["aml", "pep", "amount"]
    metadata:
      scenario: "pep_high_risk"

  - id: "aml.structuring_cash_like"
    text: "IF amount < 2000 AND tx_count_24h >= 10 AND total_amount_24h >= 15000 THEN is_suspicious = TRUE"
    description: "Wiele małych transakcji w 24h o łącznej kwocie blisko progu (structuring)."
    enabled: true
    severity: "HIGH"
    tags: ["aml", "structuring", "velocity", "amount"]
    metadata:
      scenario: "structuring"

  - id: "aml.model_score_very_high"
    text: "IF model_risk_score >= 0.9 THEN is_suspicious = TRUE"
    description: "Alert na podstawie bardzo wysokiego wyniku modelu ryzyka."
    enabled: true
    severity: "HIGH"
    tags: ["aml", "model", "score"]
    metadata:
      scenario: "model"

  - id: "aml.sanctions_match"
    text: "IF on_sanctions_list == TRUE THEN is_suspicious = TRUE"
    description: "Trafienie na liście sankcyjnej – zawsze podejrzane."
    enabled: true
    severity: "CRITICAL"
    tags: ["aml", "sanctions"]
    metadata:
      scenario: "sanctions"
"""
    aml_rules_path.write_text(aml_rules_yaml, encoding="utf-8")
    print(f"✏️ Utworzono przykładowy plik ruleset AML: {aml_rules_path}")

# Rejestracja rulesetu AML w środowisku DEV
aml_registry = RulesetRegistry()

aml_record = aml_registry.register_ruleset(
    ruleset_id="aml_rules_v1",
    path=aml_rules_path,
    environment=Environment.DEV,
)

print(
    f"📘 Zarejestrowano ruleset AML: {aml_record.key.ruleset_id} "
    f"(v{aml_record.version}) w środowisku {aml_record.key.environment.value}"
)
print(f"Plik: {aml_record.file_path}")

# Nałożenie rulesetu AML na kernel
aml_summary = aml_registry.apply_ruleset_to_kernel(
    ruleset_id="aml_rules_v1",
    environment=Environment.DEV,
    kernel=aml_kernel,
    schema=schema_aml,
    decision_field_fallback="is_suspicious",
    strict=True,
    extra_metadata={"domain": "aml-demo"},
)

print("\n📊 Podsumowanie ładowania reguł AML:")
print(f"- total_rules:   {aml_summary.total_rules}")
print(f"- enabled_rules: {aml_summary.enabled_rules}")
print(f"- loaded_rules:  {aml_summary.loaded_rules}")
print(f"- skipped_rules: {aml_summary.skipped_rules}")
print(f"- errors:        {aml_summary.errors}")


📘 Zarejestrowano ruleset AML: aml_rules_v1 (v1.0.0) w środowisku DEV
Plik: rules/aml_rules_v1.yaml

📊 Podsumowanie ładowania reguł AML:
- total_rules:   7
- enabled_rules: 7
- loaded_rules:  7
- skipped_rules: 0
- errors:        {}


In [4]:
# Cell: Weryfikacja spójności rulesetu AML w Z3

from pathlib import Path

from axiomatic_kernel import AxiomKernel
from rules_io import load_ruleset_from_file, apply_ruleset_to_kernel

# --- 1) Ustalenie ścieżki do rulesetu i schematu -----------------------------
aml_schema = schema_aml
# Jeśli masz już zmienną aml_rules_path – użyj jej.
# W przeciwnym razie zakładamy domyślną nazwę pliku.
if "aml_rules_path" in globals():
    ruleset_path = aml_rules_path
else:
    ruleset_path = Path("rules") / "aml_rules_v1.yaml"

# Schemat: preferujemy aml_schema, a jeśli go nie ma – używamy schema.
if "aml_schema" in globals():
    schema_for_check = aml_schema
elif "schema" in globals():
    schema_for_check = schema
else:
    raise RuntimeError(
        "Brak zmiennej 'aml_schema' ani 'schema' – "
        "najpierw zdefiniuj schemat zmiennych dla AML."
    )

# --- 2) Wczytanie rulesetu z YAML --------------------------------------------

ruleset = load_ruleset_from_file(ruleset_path)

# Tworzymy osobny kernel tylko do walidacji spójności.
check_kernel = AxiomKernel(
    schema=schema_for_check,
    decision_variable="is_suspicious",
    logger=None,
    rule_version=f"{ruleset.ruleset_id}:{ruleset.version}:consistency_check",
)

# --- 3) Próba nałożenia rulesetu na kernel z pełną walidacją -----------------
# Ustawiamy strict=False, żeby zebrać WSZYSTKIE błędy zamiast zatrzymać się
# na pierwszym. Jeżeli cokolwiek jest niespójne (konflikt, zły typ, zła reguła),
# pojawi się w summary.errors.

summary = apply_ruleset_to_kernel(
    kernel=check_kernel,
    ruleset=ruleset,
    schema=schema_for_check,
    decision_field_fallback="is_suspicious",
    strict=False,
)

print("=== WALIDACJA SPÓJNOŚCI RULESETU AML ===")
print(f"Plik:       {ruleset_path}")
print(f"ruleset_id: {ruleset.ruleset_id}")
print(f"version:    {ruleset.version}\n")

print("Podsumowanie ładowania reguł:")
print(f"- total_rules:   {summary.total_rules}")
print(f"- enabled_rules: {summary.enabled_rules}")
print(f"- loaded_rules:  {summary.loaded_rules}")
print(f"- skipped_rules: {summary.skipped_rules}")

if summary.errors:
    print("\n❌ Wykryto problemy w następujących regułach:")
    for rule_id, err in summary.errors.items():
        print(f"  - {rule_id}: {err}")
    print(
        "\nW szczególności komunikaty typu 'AxiomConflictError', "
        "'self-inconsistent' lub 'conflict' oznaczają logicznie "
        "sprzeczne reguły (UNSAT w Z3)."
    )
else:
    print(
        "\n✅ Wszystkie reguły zostały poprawnie załadowane do kernela.\n"
        "   AxiomKernel nie zgłosił konfliktów ani niespójnych reguł.\n"
        "   Ruleset AML jest logicznie spójny względem Z3 "
        "(w sensie add_axiom_safe)."
    )


=== WALIDACJA SPÓJNOŚCI RULESETU AML ===
Plik:       rules/aml_rules_v1.yaml
ruleset_id: aml_rules_v1
version:    1.0.0

Podsumowanie ładowania reguł:
- total_rules:   7
- enabled_rules: 7
- loaded_rules:  7
- skipped_rules: 0

✅ Wszystkie reguły zostały poprawnie załadowane do kernela.
   AxiomKernel nie zgłosił konfliktów ani niespójnych reguł.
   Ruleset AML jest logicznie spójny względem Z3 (w sensie add_axiom_safe).


In [5]:
# Cell: Batch scoring AML – decyzje + wyjaśnienia
# Wejście:  data/transactions_aml_custom.csv
# Wyjście: data/transactions_aml_with_explanations.csv

import csv
from pathlib import Path

from explanation_engine import DecisionExplainer, ExplanationConfig

# Upewniamy się, że mamy katalog z danymi
if "data_dir" not in globals():
    data_dir = Path("data")
    data_dir.mkdir(exist_ok=True)
else:
    data_dir.mkdir(exist_ok=True)

input_path = data_dir / "transactions_aml_custom.csv"
output_path = data_dir / "transactions_aml_with_explanations.csv"

# Sprawdzamy, czy kernel AML jest przygotowany
if "aml_kernel" not in globals():
    raise RuntimeError(
        "Brak zmiennej 'aml_kernel'. Najpierw uruchom komórkę z konfiguracją AML "
        "(schema_aml + aml_rules_v1 + aml_kernel)."
    )

if "aml_logger" not in globals():
    raise RuntimeError(
        "Brak zmiennej 'aml_logger'. Upewnij się, że komórka z AML kernel została wykonana."
    )

if not input_path.exists():
    raise FileNotFoundError(
        f"Plik wejściowy {input_path} nie istnieje. "
        "Najpierw uruchom generator danych AML (AMLDataGenerator)."
    )

explainer = DecisionExplainer(ExplanationConfig(language="pl"))

print(f"📥 Czytam dane AML z: {input_path}")

output_rows: list[dict] = []

with input_path.open("r", newline="", encoding="utf-8") as file_in:
    reader = csv.DictReader(file_in)
    rows = list(reader)

if not rows:
    raise RuntimeError("Plik wejściowy jest pusty – brak transakcji do przetworzenia.")

for row in rows:
    # Budujemy case zgodnie ze schema_aml
    # Używamy bezpiecznych konwersji z fallbackami
    def _to_int(value: str | None) -> int:
        if value is None or value == "":
            return 0
        return int(value)

    def _to_float(value: str | None) -> float:
        if value is None or value == "":
            return 0.0
        return float(value)

    def _to_bool(value: str | None) -> bool:
        if value is None:
            return False
        return str(value).strip().lower() in {"true", "1", "yes", "y", "t"}

    case = {
        "amount": _to_int(row.get("amount")),
        "tx_count_24h": _to_int(row.get("tx_count_24h")),
        "total_amount_24h": _to_int(row.get("total_amount_24h")),
        "tx_count_7d": _to_int(row.get("tx_count_7d")),
        "total_amount_7d": _to_int(row.get("total_amount_7d")),
        "unique_counterparties_30d": _to_int(row.get("unique_counterparties_30d")),
        "model_risk_score": _to_float(row.get("model_risk_score")),
        "is_pep": _to_bool(row.get("is_pep")),
        "on_sanctions_list": _to_bool(row.get("on_sanctions_list")),
        # is_suspicious – decyzja, więc nie podajemy w facts; kernel to wyliczy
    }

    bundle = aml_kernel.evaluate(case)
    aml_logger.log(bundle)

    decision = bundle.get("decision")
    status = bundle.get("decision_status")

    active_rules_ids = [ax["id"] for ax in bundle.get("active_axioms", [])]
    active_rules_str = ",".join(active_rules_ids)

    explanation_text = explainer.explain(bundle).to_text(language="pl")

    # Bierzemy wszystkie oryginalne kolumny + dokładamy pola decyzyjne
    output_row = dict(row)
    output_row.update(
        {
            "decision": decision,
            "decision_status": status,
            "active_rules": active_rules_str,
            "explanation_pl": explanation_text,
        }
    )
    output_rows.append(output_row)

# Zapis wynikowego pliku z decyzjami i wyjaśnieniami
fieldnames = list(output_rows[0].keys())

with output_path.open("w", newline="", encoding="utf-8") as file_out:
    writer = csv.DictWriter(file_out, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(output_rows)

print(
    f"\n✅ Zapisano wynikowy plik z decyzjami i wyjaśnieniami AML: {output_path}"
)

# Podgląd pierwszych 2 wierszy
print("\n🔎 Podgląd pierwszych 2 transakcji:")
for row in output_rows[:2]:
    print("\n" + "-" * 70)
    print(
        f"TX {row.get('transaction_id', 'N/A')} | klient: {row.get('customer_id', 'N/A')} "
        f"| decyzja: {row.get('decision')} ({row.get('decision_status')})"
    )
    print("Aktywne reguły:", row.get("active_rules") or "–")
    print("Wyjaśnienie:")
    print(row.get("explanation_pl", "").strip())


📥 Czytam dane AML z: data/transactions_aml_custom.csv

✅ Zapisano wynikowy plik z decyzjami i wyjaśnieniami AML: data/transactions_aml_with_explanations.csv

🔎 Podgląd pierwszych 2 transakcji:

----------------------------------------------------------------------
TX T000000001 | klient: C000001 | decyzja: FLAGGED (SAT)
Aktywne reguły: aml.high_amount_core
Wyjaśnienie:
Decyzja: transakcja została OFLAGOWANA (FLAGGED). Kluczowe dane wejściowe: amount=112080, is_pep=False, model_risk_score=0.72, on_sanctions_list=False, total_amount_24h=211353.

Powody (aktywne reguły):
- Reguła 'aml.high_amount_core': IF amount >= 15000 THEN is_suspicious = TRUE

Reguły, które nie zadziałały w tym przypadku:
- Reguła 'aml.velocity_24h_core' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: IF tx_count_24h >= 10 THEN is_suspicious = TRUE
- Reguła 'aml.velocity_7d_heavy' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: IF tx_count_7d >= 50 AND total_amount_7d > 5

In [6]:
# Cell: Raport AML na podstawie pliku CSV z wyjaśnieniami (AML rules)
# oraz zapis tego raportu do pliku TXT

import csv
from pathlib import Path

# Upewniamy się, że mamy katalog z danymi
if "data_dir" not in globals():
    data_dir = Path("data")

csv_path = data_dir / "transactions_aml_with_explanations.csv"
report_path = data_dir / "aml_rules_report.txt"

print(f"📄 Wczytuję raport z: {csv_path}")

# Ten string będziemy wypełniać treścią raportu, a na końcu zapiszemy do pliku TXT
report_text = ""

if not csv_path.exists():
    msg = (
        "❌ Plik transactions_aml_with_explanations.csv nie istnieje. "
        "Najpierw uruchom komórkę z batchem AML (scoring + wyjaśnienia)."
    )
    print(msg)
    report_text += msg + "\n"
else:
    with csv_path.open("r", newline="", encoding="utf-8") as file_in:
        reader = csv.DictReader(file_in)
        rows = list(reader)

    if not rows:
        msg = "Plik jest pusty, brak transakcji do raportu."
        # print(msg)
        report_text += msg + "\n"
    else:
        header = f"=== RAPORT AML (ruleset AML) – {len(rows)} transakcji ===\n\n"
        # print(header)
        report_text += header

        for idx, row in enumerate(rows, start=1):
            # --- Parsowanie pól, z bezpiecznymi fallbackami ---

            def _to_int(value: str | None) -> str | int:
                if value is None:
                    return "N/A"
                try:
                    return int(value)
                except Exception:
                    return value

            def _to_float(value: str | None) -> str | float:
                if value is None:
                    return "N/A"
                try:
                    return float(value)
                except Exception:
                    return value

            amount = _to_int(row.get("amount"))
            tx_count_24h = _to_int(row.get("tx_count_24h"))
            total_amount_24h = _to_int(row.get("total_amount_24h"))
            tx_count_7d = _to_int(row.get("tx_count_7d"))
            total_amount_7d = _to_int(row.get("total_amount_7d"))
            unique_cp_30d = _to_int(row.get("unique_counterparties_30d"))

            currency = row.get("currency", "N/A")
            kyc_risk_level = row.get("kyc_risk_level", "N/A")
            customer_segment = row.get("customer_segment", "N/A")

            is_pep_raw = str(row.get("is_pep", "")).strip()
            is_pep_bool = is_pep_raw.lower() in {"true", "1", "yes", "y", "t"}

            sanc_raw = str(row.get("on_sanctions_list", "")).strip()
            sanc_bool = sanc_raw.lower() in {"true", "1", "yes", "y", "t"}

            model_score = _to_float(row.get("model_risk_score"))

            decision = row.get("decision", "")
            status = row.get("decision_status", "")
            active_rules = row.get("active_rules") or ""
            explanation_text = row.get("explanation_pl", "").strip()

            # --- Formatowanie bloku raportu ---

            block: list[str] = []
            block.append("=" * 80)
            block.append(
                f"📌 Transakcja {row.get('transaction_id', 'N/A')} "
                f"| Klient: {row.get('customer_id', 'N/A')} "
                f"| Konto: {row.get('account_id', 'N/A')}"
            )
            block.append(
                f"Kwota: {amount} {currency} | 24h: "
                f"{tx_count_24h} tx / {total_amount_24h} | "
                f"7d: {tx_count_7d} tx / {total_amount_7d}"
            )
            block.append(
                f"Unikalni kontrahenci 30d: {unique_cp_30d} | "
                f"KYC risk: {kyc_risk_level} | Segment: {customer_segment}"
            )
            block.append(
                f"PEP: {is_pep_bool} (raw: {is_pep_raw}) | "
                f"Na liście sankcyjnej: {sanc_bool} (raw: {sanc_raw}) | "
                f"Model risk score: {model_score}"
            )
            block.append(f"Decyzja silnika: {decision} (status: {status})")

            if active_rules:
                block.append(f"Aktywne reguły (jądro decyzji): {active_rules}")
            else:
                block.append(
                    "Aktywne reguły: brak – żadna reguła nie została uruchomiona "
                    "(case CLEAN lub decyzja oparta tylko na domyślnych założeniach)."
                )

            block.append("\nUzasadnienie (pełne wyjaśnienie z DecisionExplainer):")
            block.append(explanation_text or "(brak wyjaśnienia w pliku)")
            block.append("")  # pusta linia

            block_text = "\n".join(block)
            # print(block_text)
            report_text += block_text + "\n"

        footer = "\n" + "=" * 80 + "\n🏁 Koniec raportu AML (ruleset AML).\n"
        # print(footer)
        report_text += footer

# Zapis do pliku TXT
with report_path.open("w", encoding="utf-8") as file_out:
    file_out.write(report_text)

print(f"\n💾 Zapisano raport AML do pliku: {report_path}")


📄 Wczytuję raport z: data/transactions_aml_with_explanations.csv

💾 Zapisano raport AML do pliku: data/aml_rules_report.txt
